<a target="_blank" href="https://colab.research.google.com/github/bitshrine/eCLIPsed/blob/main/notebooks/train_eclipsed.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 0. Setup

In [ ]:
import os

pkg_name = 'eCLIPsed'

%pip install git+https://github.com/openai/CLIP.git

if pkg_name not in os.listdir() or pkg_name not in os.getcwd():
    !git clone https://github.com/bitshrine/eCLIPsed/

if (pkg_name not in os.getcwd()):
    %cd eCLIPsed/
%pip install -r requirements.txt

In [ ]:
!git pull

In [3]:
from models.stylegan3.torch_utils.ops import bias_act
from models.stylegan3.torch_utils.ops import upfirdn2d

bias_act._init() and upfirdn2d._init()

Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.


True

In [4]:
%load_ext autoreload
%autoreload 2

import torch
from matplotlib import pyplot as plt
import torch

from utils.prompt_utils import *

In [5]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

# 1. Training parameters

In [6]:
from utils.train_utils import DEFAULT_TRAINING_PARAMS
TRAINING_PARAMS = DEFAULT_TRAINING_PARAMS.copy()

TRAINING_PARAMS.update({
    'latent_spaces': ['W+'],
    'loss_functions': ['L2'],
    'mask_aggregations': ['average'],
    'nums_latent_dirs': [1],
    'device': 'cuda'
})

# Since we'll be using FFHQ and Face-Bisenet, we download them here
!wget https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl -P pretrained
!gdown --id 154JgKpzCPW82qINcVieuPH3fZ2e0P812 -O pretrained/face_bisenet.pth

--2023-06-02 16:49:03--  https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl
Resolving nvlabs-fi-cdn.nvidia.com (nvlabs-fi-cdn.nvidia.com)... 65.8.248.73, 65.8.248.127, 65.8.248.59, ...
Connecting to nvlabs-fi-cdn.nvidia.com (nvlabs-fi-cdn.nvidia.com)|65.8.248.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 381624121 (364M) [binary/octet-stream]
Saving to: ‘pretrained/ffhq.pkl’

ffhq.pkl            100%[===================>] 363.94M   115MB/s    in 3.5s    

2023-06-02 16:49:08 (103 MB/s) - ‘pretrained/ffhq.pkl’ saved [381624121/381624121]

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=154JgKpzCPW82qINcVieuPH3fZ2e0P812
To: /content/eCLIPsed/pretrained/face_bisenet.pth
100% 53.3M/53.3M [00:00<00:00, 174MB/s]


In [7]:
analyzer = PromptAnalyzer(TRAINING_PARAMS)

In [38]:
prompt = "A photo of a person with curly hair\"" #@param {type:"string"}


In [39]:
part_idx, new_prompt, prompt_dir = analyzer(prompt, print_analysis=False, expansion_factor=5)

print(f'New training prompt:\n{new_prompt}')

New training prompt:
A photo a person curly hair curly hair curly hair curly hair curly hair


In [10]:
use_expanded_prompt = False #@param {type:"boolean"}

if (not use_expanded_prompt):
  analyzer.new_prompt = prompt

In [40]:
%cd /content/eCLIPsed
from utils.train_utils import train_eCLIPsed_model
torch.cuda.empty_cache()
train_eCLIPsed_model(TRAINING_PARAMS,
                analyzer=analyzer,
                localization_loss_lambda=0.08,
                gamma_correlation=0.0,
                single_image_seed=None, snapshot_interval=30)

/content/eCLIPsed


100%|██████████| 200/200 [02:50<00:00,  1.17it/s]


In [14]:
download_models = False #@param {type: "boolean"}

if (download_models):
    !zip -r /content/eCLIPsed/out.zip /content/eCLIPsed/out
    from google.colab import files
    files.download('/content/eCLIPsed/out.zip')

# 2. Running a Streamlit app

In [ ]:
!git pull

In [41]:
torch.cuda.empty_cache()

In [ ]:
app_name = "sequential_editing" #@param ["sequential_editing", "2D_visualization"]

%cd /content/eCLIPsed/apps
if (app_name == 'sequential_editing'):
  !streamlit run sequential_editing.py & npx localtunnel --port 8501
elif (app_name == '2D_visualization'):
  !streamlit run 2D_visualization.py & npx localtunnel --port 8501